Building a simple cnn Arcchitecture

In [1]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# --- Define Model Parameters ---
# Let's assume our images will be resized to 64x64 pixels with 3 color channels (RGB)
IMG_HEIGHT = 64
IMG_WIDTH = 64
# Let's assume we have 10 fashion categories to predict
NUM_CLASSES = 10

# --- Build the CNN Model ---
model = keras.Sequential([
    # Input Layer: Specify the shape of our images
    layers.Input(shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    
    # First Convolutional Block 
    layers.Conv2D(32, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    
    # Second Convolutional Block
    layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    layers.MaxPooling2D(pool_size=(2, 2)),
    
    # Flatten the 2D feature maps into a 1D vector
    layers.Flatten(),
    
    # A standard Dense layer for classification
    layers.Dense(128, activation='relu'),
    
    # Output Layer: NUM_CLASSES neurons with softmax for multi-class probability
    layers.Dense(NUM_CLASSES, activation='softmax')
])

# Print the model's summary to see how the data shape changes
model.summary()




Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 62, 62, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 31, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 29, 29, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 14, 14, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 12544)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │     1,605,760 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,626,442 (6.20 MB)

 Trainable params: 1,626,442 (6.20 MB)

 Non-trainable params: 0 (0.00 B)

Formular for Convolutional layer = (input - kernel size + 1)
Formular for Max pooling layer = floor(input/pool size)

#### CNN Image classification

In [2]:
import os
import zipfile
import shutil
import pandas as pd

# ---------------- CONFIG ----------------
CLASSES = ["Garment Lower body", "Socks & Tights", "Underwear"]  # Classes to include
N_PER_CLASS = 10  # Number of images per class
ARTICLES_CSV = "../data/articles.csv"  # Path to your CSV file
ZIP_PATH = "../data/images.zip"  # Path to downloaded zip file
OUTPUT_DIR = "../data/cnn_data/"  # Directory where extracted images will be saved
# ----------------------------------------

# Create output directories for each class
os.makedirs(OUTPUT_DIR, exist_ok=True)  # Create root output folder if it doesn't exist
for c in CLASSES:
    os.makedirs(os.path.join(OUTPUT_DIR, c), exist_ok=True)  # Create subfolder per class

# Load CSV and filter for chosen classes
df = pd.read_csv(ARTICLES_CSV)  # Read articles CSV
df['product_group_name'] = df['product_group_name'].str.strip()  # Remove extra spaces
filtered = df[df['product_group_name'].isin(CLASSES)]  # Keep only rows for selected classes

# Select top N_PER_CLASS articles per class
selected = pd.DataFrame()  # Empty DataFrame to store selected articles
for c in CLASSES:
    subset = filtered[filtered['product_group_name'] == c].head(N_PER_CLASS)  # Take first N_PER_CLASS
    selected = pd.concat([selected, subset])  # Add to selection
selected = selected.reset_index(drop=True)  # Reset row indices

# Zero-pad article_ids to 10 digits to match zip filenames
selected['article_id'] = selected['article_id'].astype(str).str.zfill(10)

# Map article_id to its class
article_class_map = dict(zip(selected['article_id'], selected['product_group_name']))  # Dict: id -> class

# Set of article_ids we need
needed_ids = set(selected['article_id'])  # For quick lookup

# ----------------- Extraction -----------------
with zipfile.ZipFile(ZIP_PATH, 'r') as z:  # Open the zip file
    for member in z.namelist():  # Loop over all files inside the zip
        if not member.lower().endswith('.jpg'):
            continue  # Skip non-image files
        article_id = os.path.splitext(os.path.basename(member))[0]  # Get article ID from filename
        if article_id in needed_ids:  # Only extract if we need it
            cls = article_class_map[article_id]  # Get the class of this article
            dst = os.path.join(OUTPUT_DIR, cls, f"{article_id}.jpg")  # Destination path
            os.makedirs(os.path.dirname(dst), exist_ok=True)  # Ensure folder exists
            with z.open(member) as source, open(dst, "wb") as target:  # Open source & destination
                shutil.copyfileobj(source, target)  # Copy image content

# ----------------- Report -----------------
for c in CLASSES:
    cnt = len([f for f in os.listdir(os.path.join(OUTPUT_DIR, c)) if f.lower().endswith('.jpg')])
    print(f"{c}: {cnt} images copied to {os.path.join(OUTPUT_DIR, c)}")  # Print how many images copied per class


Garment Lower body: 10 images copied to ../data/cnn_data/Garment Lower body
Socks & Tights: 10 images copied to ../data/cnn_data/Socks & Tights
Underwear: 10 images copied to ../data/cnn_data/Underwear


#### Preparing dataset for training,validation and also parameters


In [3]:
import tensorflow as tf  # Import TensorFlow library

# ---------------- Image & dataset parameters ----------------
IMG_HEIGHT = 64  # Height to resize all images to
IMG_WIDTH = 64   # Width to resize all images to
BATCH_SIZE = 32  # Number of images per batch during training
DATA_DIR = '../data/cnn_data/'  # Root folder where your class subfolders are

# ---------------- Create the training dataset ----------------
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,               # Directory containing subfolders per class
    validation_split=0.2,   # Reserve 20% of data for validation
    subset="training",      # Specify this dataset is the training portion
    seed=123,               # Random seed for reproducibility (ensures same split every run)
    image_size=(IMG_HEIGHT, IMG_WIDTH),  # Resize all images to uniform size
    batch_size=BATCH_SIZE   # Number of images per batch
)

# ---------------- Create the validation dataset ----------------
val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,               # Same directory as training
    validation_split=0.2,   # Same split percentage
    subset="validation",    # Specify this dataset is the validation portion
    seed=123,               # Same seed to ensure split matches training
    image_size=(IMG_HEIGHT, IMG_WIDTH),  # Resize images to same size as training
    batch_size=BATCH_SIZE   # Same batch size as training
)


Found 30 files belonging to 3 classes.
Using 24 files for training.
Found 30 files belonging to 3 classes.
Using 6 files for validation.


#### Preparation and Training

In [8]:
model.compile(
    optimizer='adam',  # Specifies the optimization algorithm used to update model weights (Adam is fast and commonly used)
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),  
        # Defines the loss function to measure how well the model predicts labels
        # 'SparseCategoricalCrossentropy' is used when labels are integers (not one-hot encoded)
        # 'from_logits=True' means the model's output layer does NOT have a softmax; loss will apply softmax internally
    metrics=['accuracy']  # Metric to track during training; here we monitor accuracy of predictions
)


print("Starting CNN training...")  
# Displays a message indicating that model training is about to begin

history = model.fit(
    train_ds,               # The training dataset containing input images and labels
    validation_data=val_ds, # The validation dataset used to evaluate performance after each epoch
    epochs=5                # Number of times the model will go through the entire training dataset
)
# 'model.fit' trains the CNN on the training data and tracks performance on validation data
# Returns a 'history' object containing training and validation loss and accuracy per epoch


Starting CNN training...
Epoch 1/5


c:\Users\hp\Projects\ai-fashion-recommender\venv\Lib\site-packages\keras\src\backend\tensorflow\nn.py:717: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.0000e+00 - loss: 67.4985 - val_accuracy: 0.5000 - val_loss: 11.6899
Epoch 2/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 314ms/step - accuracy: 0.3333 - loss: 16.7364 - val_accuracy: 0.3333 - val_loss: 331.4742
Epoch 3/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 251ms/step - accuracy: 0.4583 - loss: 204.2340 - val_accuracy: 0.3333 - val_loss: 475.2529
Epoch 4/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 343ms/step - accuracy: 0.3333 - loss: 376.1746 - val_accuracy: 0.3333 - val_loss: 311.9909
Epoch 5/5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step - accuracy: 0.3333 - loss: 212.8883 - val_accuracy: 0.3333 - val_loss: 191.9614


#### Boosting Performance  with Transfer Learning

In [5]:
import tensorflow as tf  # Import TensorFlow for deep learning

# Reuse constants from before
IMG_HEIGHT = 64  # Height of input images (must match pre-trained model's expected size)
IMG_WIDTH = 64   # Width of input images
NUM_CLASSES = 3  # Number of target categories (e.g., Dresses, Upper body, Underwear)

# -------------------- Task 1: Load Pre-trained Base Model --------------------
base_model = tf.keras.applications.MobileNetV2(
    input_shape=(IMG_HEIGHT, IMG_WIDTH, 3),  # Shape of input images (H, W, 3 for RGB)
    include_top=False,                       # Remove the original ImageNet classification head
    weights='imagenet'                        # Load weights pre-trained on ImageNet dataset
)  # This gives us a powerful feature extractor without retraining from scratch

# -------------------- Task 2: Freeze the Base Model --------------------
base_model.trainable = False  # Keep pre-trained weights fixed so they don’t get updated during training

# -------------------- Task 3: Build the New Model --------------------
model_transfer = tf.keras.Sequential([
    base_model,                                # Use frozen pre-trained CNN as the feature extractor
    tf.keras.layers.GlobalAveragePooling2D(),  # Convert feature maps to a single vector by averaging each feature map
    tf.keras.layers.Dense(NUM_CLASSES)         # Final dense layer to classify into our custom categories
])  # This stack lets us reuse old knowledge and just learn the mapping to our new labels

# Show model details
model_transfer.summary()  # Displays number of trainable vs non-trainable parameters

# -------------------- Task 4: Compile the Model --------------------
model_transfer.compile(
    optimizer='adam',  # Adam optimizer adapts learning rate for faster convergence
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),  # Loss for integer class labels
    metrics=['accuracy']  # Track accuracy during training
)

# -------------------- Task 5: Train the Model --------------------
epochs = 10  # More epochs since training will be faster with frozen layers
history_transfer = model_transfer.fit(
    train_ds,              # Training dataset
    epochs=epochs,         # Number of passes through the dataset
    validation_data=val_ds # Validation dataset to monitor generalization
)  # We expect faster accuracy improvement compared to training a CNN from scratch


C:\Users\hp\AppData\Local\Temp\ipykernel_20048\476013090.py:9: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = tf.keras.applications.MobileNetV2(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 2, 2, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │         3,843 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,261,827 (8.63 MB)

 Trainable params: 3,843 (15.01 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 15s 15s/step - accuracy: 0.4167 - loss: 1.7899 - val_accuracy: 0.6667 - val_loss: 1.0742
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 300ms/step - accuracy: 0.4167 - loss: 1.2979 - val_accuracy: 0.8333 - val_loss: 0.7939
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 256ms/step - accuracy: 0.5000 - loss: 0.9930 - val_accuracy: 0.8333 - val_loss: 0.7043
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 268ms/step - accuracy: 0.5833 - loss: 0.8491 - val_accuracy: 0.6667 - val_loss: 0.7082
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 263ms/step - accuracy: 0.5833 - loss: 0.7954 - val_accuracy: 0.5000 - val_loss: 0.7507
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 263ms/step - accuracy: 0.6250 - loss: 0.7793 - val_accuracy: 0.5000 - val_loss: 0.7983
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 258ms/step - accuracy: 0.6250 - loss: 0.7711 - val_accuracy: 0.5000 - val_loss: 0.8300
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 267ms/step - accuracy: 0.6250 - loss: 0.7557 - val_accuracy: 0.5000 - val_loss: 